**LV-posterior-01**

Demonstration of using Bayesian model with MCMC to estimate the posterior distribution of parameters and initial conditions for a Lotka-Volterra system.

In [5]:
using DifferentialEquations, Plots
# using PyPlot
# pyplot()
# using GR
# gr()
plotly()

┌ Info: For saving to png with the Plotly backend ORCA has to be installed.
└ @ Plots /Users/airwin/.julia/packages/Plots/cc8wh/src/backends.jl:363


Plots.PlotlyBackend()

In [2]:
# Pkg.build("GR")

First define the differential equation, find a numerical solution, and plot the data.

In [3]:
function lotka_volterra!(du, u, p, t)
  x, y = u
  α, β, δ, γ = p
  du[1] = dx = α*x - β*x*y
  du[2] = dy = -δ*y + γ*x*y
end

# Initial condition
u0 = [1.0, 1.0]

# Simulation interval and intermediary points
tspan = (0.0, 5.0)
tsteps = 0.0:0.5:5.0

# LV equation parameter. p = [α, β, δ, γ]
p = [1.5, 1.0, 3.0, 1.0]

# Setup the ODE problem, then solve
prob = ODEProblem(lotka_volterra!, u0, tspan, p)
sol = solve(prob, Tsit5());
sol_discrete = solve(prob, Tsit5(), saveat = tsteps);

Add some measurement error to the data.

In [6]:
data1 = Array(sol_discrete) .+ 0.8 * randn(size(Array(sol_discrete)));
Plots.plot(sol)
Plots.scatter!(sol_discrete.t, data1')

<!DOCTYPE html>
 
 
 Plots.jl

Set up for using the Bayes/MCMC tools

In [5]:
using Turing, Distributions, DifferentialEquations 

# Import MCMCChain, Plots, and StatsPlots for visualizations and diagnostics.
using MCMCChains, Plots, StatsPlots

# Set a seed for reproducibility.
using Random
Random.seed!(14);
using Logging
Logging.disable_logging(Logging.Warn)

Turing.setadbackend(:forwarddiff)


:forwarddiff

Write a function to describe a model for the parameters and initial conditions.

In [11]:
@model function fitlv(tsteps, data, prob1)
    σ ~ InverseGamma(2, 3) # ~ is the tilde character
    x0 ~ truncated(Normal(1.5,0.5),0.5,2.5)
    y0 ~ truncated(Normal(1.5,0.5),0.5,2.5)
    α ~ truncated(Normal(1.5,0.5),0.5,2.5)
    β ~ truncated(Normal(1.2,0.5),0,2)
    γ ~ truncated(Normal(3.0,0.5),1,4)
    δ ~ truncated(Normal(1.0,0.5),0,2)

    u0 = [x0, y0]
    p = [α,β,γ,δ]
    prob = remake(prob1, u0 = u0, p=p)
    predicted = solve(prob, Tsit5(), saveat=tsteps)

    for i = 1:length(predicted)
        data[:,i] ~ MvNormal(predicted[i], σ)
    end
end

fitlv (generic function with 1 method)

Now use the model to define a problem for Turing.

In [14]:
problem = ODEProblem(lotka_volterra!, u0, tspan, p)
model = fitlv(sol_discrete.t, data1, problem)
chain2 = sample(model, NUTS(.65), MCMCThreads(), 250, 4, progress=false) # not enough iterations; demo only


Chains MCMC chain (250×19×4 Array{Float64,3}):

Iterations        = 1:250
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 250
parameters        = x0, y0, α, β, γ, δ, σ
internals         = acceptance_rate, hamiltonian_energy, hamiltonian_energy_error, is_accept, log_density, lp, max_hamiltonian_energy_error, n_steps, nom_step_size, numerical_error, step_size, tree_depth

Summary Statistics
  parameters      mean       std   naive_se      mcse        ess      rhat 
      Symbol   Float64   Float64    Float64   Float64    Float64   Float64 

          x0    1.1408    0.2547     0.0081    0.0094   538.8980    1.0038
          y0    1.3982    0.3669     0.0116    0.0189   372.9874    1.0092
           α    1.5725    0.1998     0.0063    0.0098   410.1647    1.0061
           β    1.0633    0.2275     0.0072    0.0073   506.9112    0.9999
           γ    3.0315    0.4312     0.0136    0.0357   198.6961    1.0233
           δ    0.9388    0.1564     0.0049    0.0122  

Plot solution for posterior median parameters, several draws from the distribution, and the data.

In [7]:
chain_array = Array(chain2)
for k in 1:100
    u0p = chain_array[rand(1:(size(chain_array)[1])), 1:6]
    resol = solve(remake(problem, u0 = u0p[1:2], p = u0p[3:end]), Tsit5())
    Plots.plot!(resol, alpha=0.4, color = "#BBBBBB", legend = false)
end
using Statistics
u0p = median(chain_array, dims=1 )
resol = solve(remake(problem, u0 = u0p[1:2], p = u0p[3:end]), Tsit5())
Plots.plot!(sol, w=1, legend = false, lw = 2) # original solution
Plots.plot!(resol, w=1, legend = false, lw = 1) # posterior median solution
Plots.scatter!(sol_discrete.t, data1')

LoadError: [91mUndefVarError: chain2 not defined[39m